<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/pytorch-tutorial/%5BIntermediate%5D_Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [10]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# Recurrent Neural Network

'''
1. 实际上，MNIST任务用CNN这类神经网络效果更好，但这里也可以强行使用RNN/LSTM来完成任务
2. MNIST图片大小为28 * 28，可以将每张图片看作是length = 28的sequence，且序列中
每个元素的features = 28，我们将最后输出的隐藏状态hT作为抽象的隐藏特征输入全连接层进行分类
'''
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    '''
    nn.LSTM()
    - input_size: 输入特征的数目，根据前文定义这里定义为features = 28
    - hidden_size: 隐藏层的特征数目，这里定义为128
    - num_layers: 模型集成的LSTM的个数
    - bias: 是否使用bias，默认使用
    - batch_first: 默认为False，若为True，则输入/输出的格式为(batch_size, time_step, input_size)
      即[batch大小，序列长度，特征数目]。
    - dropout: 默认为0，非0则为dropout率
    - bidirectional: 是否为双向LSTM，默认为否
    '''
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    # Set initial hidden and cell states
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    '''
    - h_0 of shape(num_layers*num_directions, batch, hidden_size): tensor containing
    the [initial hidden state] for each element in the batch. If the LSTM is
    bidirectional, num_directions should be 2, else it should be 1.

    - c_0 of shape(num_layers*num_directions, batch, hidden_size): tensor containing
    the [intial cell state] for each element in the batch.

    if (h0, c0) is not provided, both h0 and c0 default to zero.
    '''

    # Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))
    '''
    out: tensor of shape (batch_size, seq_length, hidden_size)

    1. 输入格式为 lstm(input, (h0, c0))
    - 如果batch_first = True，则input = x = (batch, seq_len, input_size)，如果为False，
    则input = x = (seq_len, batch, input_size)
    - h0代表每个元素最初的隐状态值
    - c0代表每个元素最初的cell state
    2. 输出格式为 (output, (hn, cn))
    - 这里用out, _的形式进行了解包，因为我们更关注output而非hn和cn
    - output是形状为(seq_len, batch, num_directions * hidden_size)的tensor，包含输出特征
    最后输出到self.fc层中
    - hn和cn分别代表序列末尾的隐状态值和cell state
    '''

    # Decode the hidden state of the last time step
    out = self.fc(out[:, -1, :])
    return out

In [5]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.6691
Epoch [1/2], Step [200/600], Loss: 0.2543
Epoch [1/2], Step [300/600], Loss: 0.4003
Epoch [1/2], Step [400/600], Loss: 0.1512
Epoch [1/2], Step [500/600], Loss: 0.1551
Epoch [1/2], Step [600/600], Loss: 0.2048
Epoch [2/2], Step [100/600], Loss: 0.1912
Epoch [2/2], Step [200/600], Loss: 0.0758
Epoch [2/2], Step [300/600], Loss: 0.0212
Epoch [2/2], Step [400/600], Loss: 0.0673
Epoch [2/2], Step [500/600], Loss: 0.0245
Epoch [2/2], Step [600/600], Loss: 0.0541


In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97.68 %


In [9]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')